In [7]:
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from Black76 import Black76

In [10]:
df = pd.read_csv('../data/data_but.csv', sep=';')
df.drop(columns=['Unnamed: 2'], inplace=True)
df

,Strike,IV
0,33.683666,0.970740
1,33.778207,0.970035
2,33.873013,0.969329
3,33.968086,0.968623
4,34.063425,0.967917
...,...,...
995,547.740597,0.580842
996,549.277959,0.581292
997,550.819636,0.581742
998,552.365641,0.582192


## ToDo: Rewrite as a class VolSmile or Volsurface.

In [11]:
T=0.744000
F0=136.594
flag = 'c'
target_delta = 0.25

In [12]:
IV_interp = interp1d(df.Strike, df.IV)

def get_vol_from_delta(
    delta: float
):
    def BS_delta(K, flag):
        return Black76(IV_interp(K)).delta(T, K, F0, flag)

    K_target = fsolve(
        func=lambda x: BS_delta(x, flag) - delta, 
        x0=F0
    )
    return IV_interp(K_target)[0]

def risk_reversal():
    return get_vol_from_delta(0.25) - get_vol_from_delta(0.75)

def butterfly():
    return 0.5 * (get_vol_from_delta(0.25) + get_vol_from_delta(0.75)) - get_vol_from_delta(0.5)

In [13]:
print('25% Delta vol =', get_vol_from_delta(target_delta))

25% Delta vol = 0.4961605511851737


In [14]:
print('Risk reversal =', risk_reversal())
print('Butterfly =', butterfly())

Risk reversal = -0.13934496505169408
Butterfly = 0.022218984776519513
